In [1]:
import os
os.chdir("../")

In [2]:
%pwd

'c:\\Users\\USER\\Documents\\App Guardian Digital\\Guardian-Digital'

In [3]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

c:\Users\USER\anaconda3\envs\medibot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# Extrayendo texto de arhivos PDF
def load_pdf_files(data):
    loader = DirectoryLoader(
        data, 
        glob="*.pdf", 
        loader_cls=PyPDFLoader
        )
    documents = loader.load()
    return documents

In [5]:
extracted_data = load_pdf_files("data")


In [6]:
extracted_data

[Document(metadata={'producer': 'www.ilovepdf.com', 'creator': 'PyPDF', 'creationdate': '2020-06-09T13:01:58+02:00', 'moddate': '2020-09-15T13:31:37+02:00', 'source': 'data\\GPC_Conducta_Suicida.pdf', 'total_pages': 381, 'page': 0, 'page_label': '1'}, page_content='Guía de Práctica Clínica  \nde Prevención y Tratamiento \nde la Conducta Suicida\nGUÍAS DE PRÁCTICA CLÍNICA EN EL SNS\nMINISTERIO DE SANIDAD, POLÍTICA SOCIAL E IGUALDAD\nMINISTERIO \nDE CIENCIA \nE INNOVACIÓN\nM INISTERIO  DE   \nS\n \nANIDAD\n, P\n \nOLÍTICA\n \n S\n \nOCIAL\n \n \nE\n \n I\n \nGUALDAD'),
 Document(metadata={'producer': 'www.ilovepdf.com', 'creator': 'PyPDF', 'creationdate': '2020-06-09T13:01:58+02:00', 'moddate': '2020-09-15T13:31:37+02:00', 'source': 'data\\GPC_Conducta_Suicida.pdf', 'total_pages': 381, 'page': 1, 'page_label': '2'}, page_content='Guía de Práctica Clínica  \nde Prevención y Tratamiento \nde la Conducta Suicida\nGUÍAS DE PRÁCTICA CLÍNICA EN EL SNS\nMINISTERIO DE SANIDAD, POLÍTICA SOCIAL E 

In [7]:
len(extracted_data)

554

In [8]:
from typing import List
from langchain.schema import Document

def filter_to_minimal_docs(docs: List[Document]) -> List[Document]:
    minimal_docs: List[Document] = [] 
    
    for doc in docs:
        src = doc.metadata.get("source")
        minimal_docs.append(
            Document(
                page_content=doc.page_content,
                metadata={"source": src}
            )
        )
    return minimal_docs

In [9]:
minimal_docs = filter_to_minimal_docs(extracted_data)

In [10]:
minimal_docs

[Document(metadata={'source': 'data\\GPC_Conducta_Suicida.pdf'}, page_content='Guía de Práctica Clínica  \nde Prevención y Tratamiento \nde la Conducta Suicida\nGUÍAS DE PRÁCTICA CLÍNICA EN EL SNS\nMINISTERIO DE SANIDAD, POLÍTICA SOCIAL E IGUALDAD\nMINISTERIO \nDE CIENCIA \nE INNOVACIÓN\nM INISTERIO  DE   \nS\n \nANIDAD\n, P\n \nOLÍTICA\n \n S\n \nOCIAL\n \n \nE\n \n I\n \nGUALDAD'),
 Document(metadata={'source': 'data\\GPC_Conducta_Suicida.pdf'}, page_content='Guía de Práctica Clínica  \nde Prevención y Tratamiento \nde la Conducta Suicida\nGUÍAS DE PRÁCTICA CLÍNICA EN EL SNS\nMINISTERIO DE SANIDAD, POLÍTICA SOCIAL E IGUALDAD\nMINISTERIO \nDE CIENCIA \nE INNOVACIÓN\nM INISTERIO  DE   \nS\nANIDAD\n, P\nOLÍTICA\n S\nOCIAL\n \nE\n I\nGUALDAD'),
 Document(metadata={'source': 'data\\GPC_Conducta_Suicida.pdf'}, page_content='Esta GPC es una ayuda a la toma de decisiones en la atención sanitaria. No es de obligado cumplimento ni sustituye el \njuicio clínico del personal sanitario.\nEdición:

In [11]:
# Dividir los documentos en fragmentos más pequeños
def text_split(minimal_docs):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500, 
        chunk_overlap=20,
    )
    texts_chunk = text_splitter.split_documents(minimal_docs)  
    return texts_chunk

In [12]:
texts_chunk = text_split(minimal_docs)
print(f"Number of chunks: {len(texts_chunk)}")

Number of chunks: 3066


In [13]:
texts_chunk

[Document(metadata={'source': 'data\\GPC_Conducta_Suicida.pdf'}, page_content='Guía de Práctica Clínica  \nde Prevención y Tratamiento \nde la Conducta Suicida\nGUÍAS DE PRÁCTICA CLÍNICA EN EL SNS\nMINISTERIO DE SANIDAD, POLÍTICA SOCIAL E IGUALDAD\nMINISTERIO \nDE CIENCIA \nE INNOVACIÓN\nM INISTERIO  DE   \nS\n \nANIDAD\n, P\n \nOLÍTICA\n \n S\n \nOCIAL\n \n \nE\n \n I\n \nGUALDAD'),
 Document(metadata={'source': 'data\\GPC_Conducta_Suicida.pdf'}, page_content='Guía de Práctica Clínica  \nde Prevención y Tratamiento \nde la Conducta Suicida\nGUÍAS DE PRÁCTICA CLÍNICA EN EL SNS\nMINISTERIO DE SANIDAD, POLÍTICA SOCIAL E IGUALDAD\nMINISTERIO \nDE CIENCIA \nE INNOVACIÓN\nM INISTERIO  DE   \nS\nANIDAD\n, P\nOLÍTICA\n S\nOCIAL\n \nE\n I\nGUALDAD'),
 Document(metadata={'source': 'data\\GPC_Conducta_Suicida.pdf'}, page_content='Esta GPC es una ayuda a la toma de decisiones en la atención sanitaria. No es de obligado cumplimento ni sustituye el \njuicio clínico del personal sanitario.\nEdición:

In [14]:
from langchain.embeddings import HuggingFaceEmbeddings

def download_embeddings():
    model_name = "sentence-transformers/all-MiniLM-L6-v2"
    embeddings = HuggingFaceEmbeddings(
        model_name=model_name
        )
    return embeddings

embedding = download_embeddings()

C:\Users\USER\AppData\Local\Temp\ipykernel_15056\661828901.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


In [15]:
embedding

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [16]:
vector = embedding.embed_query("Hello World")
vector

[-0.03447723761200905,
 0.031023172661662102,
 0.006734994240105152,
 0.026108955964446068,
 -0.03936205431818962,
 -0.16030253469944,
 0.06692399829626083,
 -0.006441488862037659,
 -0.04745050147175789,
 0.014758973382413387,
 0.07087533175945282,
 0.05552758648991585,
 0.019193360581994057,
 -0.02625136263668537,
 -0.010109484195709229,
 -0.026940476149320602,
 0.022307496517896652,
 -0.022226696833968163,
 -0.14969263970851898,
 -0.0174931101500988,
 0.007676286157220602,
 0.05435218662023544,
 0.003254459472373128,
 0.0317259356379509,
 -0.08462143689393997,
 -0.029405932873487473,
 0.05159565806388855,
 0.04812400043010712,
 -0.0033148087095469236,
 -0.058279216289520264,
 0.041969332844018936,
 0.022210735827684402,
 0.12818878889083862,
 -0.022338945418596268,
 -0.01165622379630804,
 0.06292835623025894,
 -0.03287626802921295,
 -0.09122609347105026,
 -0.031175386160612106,
 0.052699603140354156,
 0.047034852206707,
 -0.0842030793428421,
 -0.03005618415772915,
 -0.020744716748595

In [17]:
print(f"Vector length: {len(vector)}")

Vector length: 384


In [18]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [19]:
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [20]:
from pinecone import Pinecone
pinecode_api_key = PINECONE_API_KEY

pc = Pinecone(api_key=pinecode_api_key)
pc

In [21]:
from pinecone import ServerlessSpec

index_name = "guardian-digital"

if not pc.has_index(index_name):
    pc.create_index(
        name=index_name,
        dimension=384,  # Dimensión de los vectores de embedding
        metric="cosine",  # Métrica de similitud
        spec = ServerlessSpec(cloud="aws", region="us-east-1")
    )

index = pc.Index(index_name)

In [22]:
from langchain_pinecone import PineconeVectorStore

dosearch = PineconeVectorStore.from_documents(
    documents=texts_chunk,
    embedding=embedding,
    index_name=index_name,
)

In [23]:
from langchain_pinecone import PineconeVectorStore
dosearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embedding,
)

In [24]:
retriever = dosearch.as_retriever(search_type="similarity", search_kwargs={"k": 3})

In [25]:
retriever_docs = retriever.get_relevant_documents("¿Que es un intento suicida?")
retriever_docs

C:\Users\USER\AppData\Local\Temp\ipykernel_15056\4279365566.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retriever_docs = retriever.get_relevant_documents("¿Que es un intento suicida?")


[Document(id='8580a358-9c40-497b-b54c-84747f013828', metadata={'source': 'data\\mhGAP_2.0.pdf'}, page_content='o impulsividad, o en riesgo de suicidio.'),
 Document(id='babf6fc4-4f5e-4146-8357-b453ce3ebe8a', metadata={'source': 'data\\mhGAP_2.0.pdf'}, page_content='o impulsividad, o en riesgo de suicidio.'),
 Document(id='52e49851-6a75-4b3c-8df3-265df7680bed', metadata={'source': 'data\\mhGAP_2.0.pdf'}, page_content='o impulsividad, o en riesgo de suicidio.')]

In [26]:
from langchain_openai import ChatOpenAI

chatModel = ChatOpenAI(model="gpt-4o")

In [27]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

In [40]:
system_prompt = (
    "Eres 'Guardián Digital', un asistente de apoyo psicológico basado en la guía mhGAP y protocolos de prevención del suicidio. "
    "Utiliza los siguientes fragmentos de contexto recuperado para responder a la pregunta con empatía y precisión clínica. "
    "Si no sabes la respuesta basándote en el contexto, di honestamente que no lo sabes. NO inventes información médica. "
    "ALERTA DE SEGURIDAD: Si la pregunta o el contexto sugieren riesgo de suicidio, autolesión o desesperanza extrema, "
    "Mantén la respuesta sencilla, cálida y directa. Maximo 3 a 4 lineas"
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


In [41]:
question_answer_chain = create_stuff_documents_chain(chatModel, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [30]:
response = rag_chain.invoke({"input": "tengo pensamientos suicidas"})
print(response["answer"])

Siento mucho que estés pasando por esto. Es importante que hables con alguien que pueda ayudarte, como un profesional de la salud mental o una línea de ayuda de emergencia en tu área. No estás solo, y hay personas que se preocupan y quieren apoyarte. Por favor, busca ayuda inmediatamente.


In [31]:
response = rag_chain.invoke({"input": "Cual seria mi tratamiento si tengo pensamientos suicidas? "})
print(response["answer"])

Lamento mucho que te sientas así. Es importante que sepas que las crisis son transitorias y muchas personas han logrado superarlas. Te recomiendo que hables con un profesional de salud mental que pueda ofrecerte el apoyo adecuado y que permanezcas acompañado hasta que esos pensamientos disminuyan. Si estás en peligro inmediato, busca ayuda de emergencia o contacta a servicios de crisis en tu área.


In [38]:
response = rag_chain.invoke({"input": "Hazme algunas preguntas para saber si tengo riesgo suicida"})
print(response["answer"])

Lamento que te sientas así. Es importante hablar con un profesional capacitado que pueda ayudarte. Si te sientes cómodo compartiendo, algunas preguntas podrían ser: ¿Has pensado en hacerte daño o suicidarte? ¿Tienes un plan en mente? Recuerda, no estás solo; busca apoyo de inmediato. Tu bienestar es lo más importante.


In [42]:
response = rag_chain.invoke({"input": "Hazme algunas preguntas para saber si tengo riesgo suicida"})
print(response["answer"])

Entiendo que este puede ser un tema difícil. Me gustaría que pudieras responder algunas preguntas: ¿Te has sentido abrumado o atrapado recientemente? ¿Has tenido pensamientos de querer lastimarte o no estar más aquí? ¿Te sientes desesperanzado sobre el futuro? Es importante buscar ayuda profesional si te encuentras en una situación de riesgo.
